In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.utils import rnn
import sru

In [9]:


class ConvBSRU():
    def __init__(self, frame_size, conv_channels, stride=128, num_layers=1, dropout=0.1, rescale=False, bidirectional=True):
        super(ConvBSRU, self).__init__()
        num_directions = 2 if bidirectional else 1
        if stride == frame_size:
            padding = 0
        elif stride == frame_size // 2:
            padding = frame_size // 2
        else:
            print(stride, frame_size)
            raise ValueError(
                'Invalid stride {}. Length of stride must be "frame_size" or "0.5 * "frame_size"'.format(stride))
            
        self.conv = nn.Conv1d(
            in_channels=1, 
            out_channels=conv_channels, 
            kernel_size=frame_size, 
            stride=stride,
            padding=padding,
            bias=False
        )
        self.deconv = nn.ConvTranspose1d(
            in_channels=conv_channels,
            out_channels=1,
            kernel_size=frame_size,
            stride=stride,
            padding=padding,
            bias=False
        )
        self.outfc = nn.Linear(num_directions * conv_channels, conv_channels, bias=False)
        self.sru1 = sru.SRU(
            input_size=conv_channels,
            hidden_size=conv_channels,
            num_layers=num_layers,
            dropout=dropout,
            rnn_dropout=0.1,
            layer_norm=True,
            rescale=rescale,
            bidirectional=bidirectional
        )

    def forward(self, x):
        output = self.conv(x)
        
        output_ = self.sru1(output)
        
        output_ = self.outdc(ouput_)
        
        output = output * output_
        
        output = self.deconv(output)
        
        return output